### Enable GPU Acceleration

In [1]:
from numba import cuda
import numpy as np
 
@cuda.jit
def add_kernel(x, y, out):
    idx = cuda.grid(1)
    out[idx] = x[idx] + y[idx]
 
n = 4096
x = np.arange(n).astype(np.int32) # [0...4095] on the host
y = np.ones_like(x)               # [1...1] on the host
 
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
out = np.zeros_like(x)
d_out = cuda.to_device(out)
 
threads_per_block = 128
blocks_per_grid = 32
 
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host()) # Should be [1...4096]

[   1    2    3 ... 4094 4095 4096]


### Import Packages

In [2]:
import pandas as pd # data manipulation
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np #  mathematical support for large, multi-dimensional arrays and matrices
import os

In [3]:
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-1k1gc7b3 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import re

### Set Directory

In [5]:
os.getcwd()

'/scratch/interns202010/jmakins/jupyter-dir'

In [6]:
try:
    os.chdir('/group/interns202010/jmakins/Data')
except:
    pass

In [7]:
os.listdir()

['world_soccer_database.sqlite',
 'events',
 'referees.json',
 'players.json',
 'competitions.json',
 'teams.json',
 'coaches.json',
 'matches',
 '.DS_Store']

In [8]:
Teams = pd.read_json('teams.json')
Competitions = pd.read_json('competitions.json')
Players = pd.read_json('players.json')
Coaches = pd.read_json('coaches.json') # Managers

In [9]:
#Competitions['CompId'] = Competitions['wyId']

In [10]:
#Competitions.drop(['wyId', 'CountryId'], inplace = True, axis = 1)

In [11]:
try:
    os.chdir('/group/interns202010/jmakins/Data/matches')
except:
    pass
England = pd.read_json('matches_England.json')
France = pd.read_json('matches_France.json')
Italy = pd.read_json('matches_Italy.json')
Spain = pd.read_json('matches_Spain.json')
Germany = pd.read_json('matches_Germany.json')
World_Cup = pd.read_json('matches_World_Cup.json')
Euro_Champs = pd.read_json('matches_European_Championship.json')

In [12]:
# Adding country labels to match observations
England['Country'] = 'England'
France['Country'] = 'France'
Italy['Country'] = 'Italy'
Spain['Country'] = 'Spain'
Germany['Country'] = "Germany"
World_Cup['Country'] = 'World Cup'
Euro_Champs['Country'] = 'European Champs'

In [13]:
try:
    os.chdir('/group/interns202010/jmakins/Data/events')
except:
    pass

In [14]:
os.listdir()

['events_Spain.json',
 'events_Germany.json',
 'events_France.json',
 'events_European_Championship.json',
 'events_England.json',
 'events_World_Cup.json',
 '.DS_Store',
 'events_Italy.json']

In [15]:
Events_France = pd.read_json('events_France.json')
Events_Spain = pd.read_json('events_Spain.json')
Events_Germany = pd.read_json('events_Germany.json')
Events_EuroChamps = pd.read_json('events_European_Championship.json')
Events_World_Cup = pd.read_json('events_World_Cup.json')
Events_Italy = pd.read_json('events_Italy.json')
Events_England = pd.read_json('events_England.json')

In [16]:
# Example of an events table
Events_England[Events_England['matchId'] == 2499719]

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
...,...,...,...,...,...,...,...,...,...,...,...,...
1763,8,Launch,[{'id': 1802}],8480,"[{'y': 53, 'x': 7}, {'y': 49, 'x': 46}]",2499719,Pass,1631,2H,2992.491575,84,177961040
1764,8,Head pass,[{'id': 1801}],49876,"[{'y': 51, 'x': 54}, {'y': 58, 'x': 73}]",2499719,Pass,1609,2H,2994.900590,82,177961035
1765,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1216}, {'id'...",7870,"[{'y': 58, 'x': 73}, {'y': 0, 'x': 0}]",2499719,Shot,1609,2H,2997.086392,100,177961036
1766,5,Ball out of the field,[],0,"[{'y': 48, 'x': 3}, {'y': 100, 'x': 100}]",2499719,Interruption,1631,2H,2998.963701,50,177961041


### Data Cleaning

In [17]:
Matches = pd.concat([England, France, Italy, Spain, Germany, World_Cup, Euro_Champs], ignore_index=True)

In [18]:
Matches.drop(["groupName", 'duration'], inplace=True, axis =1) # removing groupName variable from World Cups

In [19]:
Players.drop(["passportArea"], inplace = True, axis = 1)

In [20]:
# Example of Players dataframe
Players[0:3]

,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null


In [21]:
Players['Position'] = Players.role.apply(pd.Series)['code3'].values

In [22]:
Players['birthCountry'] = Players.birthArea.apply(pd.Series)['name'].values

In [23]:
Players.drop(["birthArea", 'role'], inplace = True, axis = 1)

### Manipulate Match Data

In [24]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match

In [25]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for x in teams:
    homeTeam.append(x[0])
    awayTeam.append(x[1])

In [26]:
# add columns for additional match statistics
Matches["homeManagerId"] = None
for name in ["Score", "ScoreHT", "ScoreET", "ScoreP"]:
    colname = "home" + name
    colname2 = "away" + name
    Matches[colname] = None
    Matches[colname2] = None

In [27]:
# parse Home and Away teams as features from nest dictionary
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for game, team in enumerate(teams):
    if Matches.teamsData[game:game+1][game][team[0]]['side'] == 'home':
        homeTeam.append(team[0])
        awayTeam.append(team[1])
    else:
        awayTeam.append(team[0])
        homeTeam.append(team[1])

In [28]:
def get_team_stats(df, team): # function to parse team match stats from dictionary embedded in column teamsData from the Matches dataframe
    
    Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId = [],[],[],[],[],[],[],[]
    
    for game in range(0, len(df)):

        Bench.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['bench'])['playerId']))
        Starters.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['lineup'])['playerId']))
        Manager.append(df.teamsData[game][team[game]]['coachId'])
        Score.append(df.teamsData[game][team[game]]['score'])
        ScoreHT.append(df.teamsData[game][team[game]]['scoreHT'])
        ScoreET.append(df.teamsData[game][team[game]]['scoreET'])
        ScoreP.append(df.teamsData[game][team[game]]['scoreP'])
        teamId.append(df.teamsData[game][team[game]]['teamId'])
    
    return Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId

homeBench, homeStarters, homeManager, homeScore, homeScoreHT, homeScoreET, homeScoreP, homeTeamId = get_team_stats(Matches, homeTeam)
awayBench, awayStarters, awayManager, awayScore, awayScoreHT, awayScoreET, awayScoreP, awayTeamId = get_team_stats(Matches, awayTeam)


In [29]:
# Automating Bench Line Up Column Generation

homeColNames = []
for num in range(1,14): # add columns for home bench playerId
     homeColNames.append("homeBenchPlayer" + str(num))
        
awayColNames = []
for num in range(1,14): # add columns for away bench playerId
    awayColNames.append("awayBenchPlayer" + str(num))
    


In [30]:
# Inspecting bench counts as during errror handling reveals some matches had bench sizes of between 8 and 13 for certain fixtures
import collections
collections.Counter(list(map(lambda x: len(x), homeBench)))

Counter({7: 1425, 6: 18, 5: 1, 8: 5, 12: 292, 10: 53, 11: 123, 9: 23, 13: 1})

In [31]:
# Adding Bench Line Ups to DataFrame

homeBench2 = pd.DataFrame(homeBench,
     columns=homeColNames)
awayBench2 = pd.DataFrame(awayBench,
     columns=awayColNames)


In [32]:
# Automating Starting Line Up Column Generation

homeColNames =[]
for num in range(1,12): # add columns for starting playerId
     homeColNames.append("homePlayer" + str(num))
        
awayColNames = []
for num in range(1,12): # add columns for starting playerId
    awayColNames.append("awayPlayer" + str(num))


In [33]:
# Joining starting line ups, benches to matches dataframe

homeStarters2 = pd.DataFrame(homeStarters,
     columns=homeColNames)
awayStarters2 = pd.DataFrame(homeStarters,
     columns=awayColNames)

Matches = Matches.join([homeStarters2, awayStarters2, homeBench2, awayBench2])


In [34]:
# add individual columns of match data
Matches['homeManager'] = homeManager
Matches['homeScore'] = homeScore
Matches['homeScoreHT'] = homeScoreHT
Matches['homeScoreET'] = homeScoreET
Matches['homeScoreP'] = homeScoreP
Matches['homeTeamId'] = homeTeamId
Matches['awayManager'] = awayManager
Matches['awayScore'] = awayScore
Matches['awayScoreHT'] = awayScoreHT
Matches['awayScoreET'] = awayScoreET
Matches['awayScoreP'] = awayScoreP
Matches['awayTeamId'] = awayTeamId

In [35]:
Matches.drop(["teamsData"], inplace = True, axis = 1)

In [36]:
# Classify Match Result as W, L or D for teams
conditions = [
    (Matches['winner'] == 0),
    (Matches['winner'] == Matches['awayTeamId']),
    (Matches['winner'] == Matches['homeTeamId'])
    ]

# create a list of the values we want to assign for each condition
values = [0, -1, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
Matches['Result'] = np.select(conditions, values)

###### Relating table to access Referee Data

In [37]:
# os.chdir('/group/interns202010/jmakins/Data')

In [38]:

# sql_conn = sqlite3.connect('world_soccer_database.sqlite') # soccer database containing over 100,000 match results which is updated monthly

# Match_TeamStats = pd.read_sql_query("SELECT * FROM football_data", sql_conn)

# # parsing betting odds variables from analysis
# drop_columns = ["B365H","B365D","B365A","BSH","BSD","BSA","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA",
# "PSH", "PH","PSD", "PD","PSA", "PA","SOH","SOD","SOA","SBH","SBD","SBA","SJH","SJD","SJA","SYH","SYD","SYA","VCH",
# "VCD","VCA","WHH","WHD","WHA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","MaxH","MaxD","MaxA",
# "AvgH","AvgD","AvgA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","GB>2.5","GB<2.5","B365>2.5","B365<2.5",
# "P>2.5","P<2.5","Max>2.5","Max<2.5","Avg>2.5","Avg<2.5","BbAH","BbAHh","AHh","BbMxAHH","BbAvAHH","BbMxAHA",
# "BbAvAHA","GBAHH","GBAHA","GBAH","LBAHH","LBAHA","LBAH","B365AHH","B365AHA","B365AH","PAHH", "AvgC<2.5",
# "PAHA","MaxAHH","MaxAHA","AvgAHH","AvgAHA","WHCH","WHCD","WHCA", "VCCH", "VCCA", "PSCH", "PSCD", "PCAHH", "PCAHA", "PC>2.5", "PC<2.5", "MaxCH", "MaxCAHH", "MaxCD",
# "MaxCAHA", "MaxCA", "MaxC>2.5", "MaxC<2.5", "IWCH", "IWCD", "IWCA", "AvgC>2.5","AvgCA","AvgCAHA","AvgCAHH",
# "AvgCD","AvgCH","B365C<2.5","B365C>2.5","B365CA","B365CAHA","B365CAHH","B365CD","B365CH","BWCA", "BWCD", "BWCH", "VCCD", "PSCA"] 

# Match_TeamStats.drop(drop_columns, inplace=True, axis =1) # removing betting odds from dataframe





In [39]:
# Match_TeamStats = Match_TeamStats[Match_TeamStats['Datetime'].str.contains('2017|2018')]

In [40]:
Matches["date"] = pd.to_datetime(Matches['dateutc']).dt.date # create a date column for Matches dataframe

In [41]:
Matches["time"] = pd.to_datetime(Matches['dateutc']).dt.time # create a time column for Matches dataframe

In [43]:
# Dimensionality Reduction
cols2Drop = ['status', 'roundId', 'gameweek', 'dateutc', 'label', 'referees', 'homeManagerId', 'seasonId']
benches2Drop = []
for benchP in range(8,14):
    benches2Drop.append("awayBenchPlayer" + str(benchP))
    benches2Drop.append("homeBenchPlayer" + str(benchP))
    
cols2Drop = cols2Drop + benches2Drop

In [44]:
Matches.drop(cols2Drop, inplace=True, axis =1)

In [ ]:
# Match_TeamStats['date'] = pd.to_datetime(Match_TeamStats['Datetime']).dt.date

In [ ]:
# # Get Referee and Home/Away Team Information
# Match_TeamStats.merge(Matches, how = "inner", indicator = False)

In [ ]:
# Matches = Match_TeamStats.merge(Matches, how = "inner", indicator = False)

In [ ]:
# Matches.drop(['gameweek', 'seasonId', 'dateutc', 'label', 'Season', 'status', 'roundId'], inplace= True, axis =1)

In [ ]:
# Matches.drop(['Datetime', 'Div','FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'ABP', 'AC',
#        'AF', 'AFKC', 'AHCh', 'AHW', 'AO', 'AR', 'AS', 'AST', 'AT', 'AY',
#        'Attendance', 'Date', 'HBP', 'HC', 'HF', 'HFKC', 'HHW', 'HO', 'HR',
#        'HS', 'HST', 'HT', 'HY', 'Time', 'referees'], inplace= True, axis =1)

### Events Manipulation for Match/Player Stats and then combine with existing Match(test) dataframe from aboves

In [ ]:
# Group by and events summarization

In [45]:
Events_England['subEventName'].unique()

array(['Simple pass', 'High pass', 'Head pass', 'Air duel',
       'Ground loose ball duel', 'Smart pass', 'Launch',
       'Ground defending duel', 'Ground attacking duel', 'Foul',
       'Free Kick', 'Cross', 'Shot', 'Reflexes', 'Touch', 'Clearance',
       'Ball out of the field', 'Throw in', 'Goal kick', 'Corner',
       'Goalkeeper leaving line', 'Hand pass', 'Acceleration',
       'Save attempt', '', 'Free kick cross', 'Free kick shot',
       'Hand foul', 'Violent Foul', 'Protest', 'Whistle',
       'Late card foul', 'Out of game foul', 'Penalty', 'Time lost foul',
       'Simulation'], dtype=object)

In [46]:
tags = [
    (101, "goal"),
    (102, "own_goal"),
    (301, "assist"),
    (302, "key_pass"),
    (1901, "counter_attack"),
    (401, "left_foot"),
    (402, "right_foot"),
    (403, "head/body"),
    (1101, "direct"),
    (1102, "indirect"),
    (2001, "dangerous_ball_lost"),
    (2101, "blocked"),
    (801, "high"),
    (802, "low"),
    (1401, "interception"),
    (1501, "clearance"),
    (201, "opportunity"),
    (1301, "feint"),
    (1302, "missed_ball"),
    (501, "free_space_right"),
    (502, "free_space_left"),
    (503, "take_on_left"),
    (504, "take_on_right"),
    (1601, "sliding_tackle"),
    (601, "anticipated"),
    (602, "anticipation"),
    (1701, "red_card"),
    (1702, "yellow_card"),
    (1703, "second_yellow_card"),
    (1201, "position_goal_low_center"),
    (1202, "position_goal_low_right"),
    (1203, "position_goal_mid_center"),
    (1204, "position_goal_mid_left"),
    (1205, "position_goal_low_left"),
    (1206, "position_goal_mid_right"),
    (1207, "position_goal_high_center"),
    (1208, "position_goal_high_left"),
    (1209, "position_goal_high_right"),
    (1210, "position_out_low_right"),
    (1211, "position_out_mid_left"),
    (1212, "position_out_low_left"),
    (1213, "position_out_mid_right"),
    (1214, "position_out_high_center"),
    (1215, "position_out_high_left"),
    (1216, "position_out_high_right"),
    (1217, "position_post_low_right"),
    (1218, "position_post_mid_left"),
    (1219, "position_post_low_left"),
    (1220, "position_post_mid_right"),
    (1221, "position_post_high_center"),
    (1222, "position_post_high_left"),
    (1223, "position_post_high_right"),
    (901, "through"),
    (1001, "fairplay"),
    (701, "lost"),
    (702, "neutral"),
    (703, "won"),
    (1801, "accurate"),
    (1802, "not_accurate"),
]

In [47]:
tags = dict(tags)

In [48]:
# ## I have kept this cell encase we need to impute integers for ML algorithms as oppose to strings
# event_tag_ids = []
# for ids in  list(trial[0:10]['tags']):
#     event_tag_ids.append(list(map(lambda y: y['id'], ids)))
# trial['event_tag_ids'] = event_tag_ids

In [49]:
trial = Events_England[Events_England['matchId'] == 2499719]

In [50]:
# Store event type tags ids in new column
def event_tags(event_df):
    event_tags = []
    for ids in  list(event_df['tags']):
        event_tags.append(list(map(lambda y: tags[y['id']], ids)))

    event_df['event_tags'] = event_tags
    return event_df

In [51]:
Events_England = event_tags(Events_England)

In [52]:
# Clean the result of the action
def Result(event_df):
    result = []
    for tag in event_df["event_tags"]:
        if "accurate" in tag:
            result.append("Success")
        elif "not_accurate" in tag:
            result.append("Failure")
        else:
            result.append("")
    event_df["Result"] = result 
    return event_df

In [53]:
Events_England = Result(Events_England)

In [54]:
# Separating description from success/failure of an action (event)
def event_description(event_df):
    subEventDescr = []
    for tag in event_df["event_tags"]:
        descr = " ".join(tag[:-1])
        if descr != "":
            subEventDescr.append(descr)
        else:
            subEventDescr.append("generic play")
    event_df["subEventDescription"] = subEventDescr
    event_df = event_df[(event_df["event_tags"].str.len() != 0) & (event_df.subEventName != 'Ball out of the field')  & (event_df.subEventName != 'Goal kick') & (event_df.subEventName != 'Touch')   ]
    event_df = event_df[event_df["subEventName"] != "Throw in"] # remove throw-ins as a relevant feature among successful teams for simpler analysis, unless its Rory Delap!
    return event_df

In [55]:
Events_England = event_description(Events_England)

In [56]:
#### I prefer the segmented format of the cell below for locations but we will keep just encase***
# start, end = [], []
# for pos1, pos2 in trial['positions']:
#     start.append(pos1)
#     end.append(pos2)

# trial["StartLoc"] = start
# trial["EndLoc"] = end

In [57]:
collections.Counter(list(map(lambda x: len(x), Events_England['positions'])))

Counter({2: 548318, 1: 118})

In [58]:
for pos in Events_England[~(Events_England.positions.str.len() ==2)]['positions']:
    print(pos[0])

{'y': 7, 'x': 62}
{'y': 14, 'x': 70}
{'y': 70, 'x': 62}
{'y': 45, 'x': 22}
{'y': 19, 'x': 40}
{'y': 47, 'x': 45}
{'y': 6, 'x': 48}
{'y': 20, 'x': 17}
{'y': 56, 'x': 1}
{'y': 0, 'x': 58}
{'y': 86, 'x': 42}
{'y': 91, 'x': 68}
{'y': 10, 'x': 62}
{'y': 9, 'x': 3}
{'y': 25, 'x': 86}
{'y': 33, 'x': 42}
{'y': 63, 'x': 27}
{'y': 32, 'x': 56}
{'y': 52, 'x': 6}
{'y': 49, 'x': 35}
{'y': 18, 'x': 42}
{'y': 74, 'x': 75}
{'y': 62, 'x': 22}
{'y': 26, 'x': 54}
{'y': 85, 'x': 6}
{'y': 1, 'x': 20}
{'y': 33, 'x': 61}
{'y': 64, 'x': 94}
{'y': 24, 'x': 22}
{'y': 40, 'x': 69}
{'y': 89, 'x': 86}
{'y': 60, 'x': 42}
{'y': 53, 'x': 38}
{'y': 21, 'x': 61}
{'y': 9, 'x': 83}
{'y': 54, 'x': 38}
{'y': 60, 'x': 85}
{'y': 64, 'x': 61}
{'y': 27, 'x': 20}
{'y': 92, 'x': 29}
{'y': 97, 'x': 90}
{'y': 44, 'x': 61}
{'y': 98, 'x': 69}
{'y': 68, 'x': 11}
{'y': 40, 'x': 5}
{'y': 7, 'x': 25}
{'y': 70, 'x': 37}
{'y': 13, 'x': 36}
{'y': 28, 'x': 36}
{'y': 0, 'x': 99}
{'y': 76, 'x': 32}
{'y': 3, 'x': 92}
{'y': 61, 'x': 36}
{'y': 1

In [59]:
test = []
for pos in Events_England["positions"][0:10]:
    test.append(pos[0]['y'])

In [60]:
xStart,xEnd,yStart,yEnd = [], [], [], []
for pos in Events_England["positions"][0:10]:
    if len(pos) == 2:
        xStart.append(pos[0]['x'])
        xEnd.append(pos[1]['x'])
        yStart.append(pos[0]['y'])
        yEnd.append(pos[1]['y'])
    else:
        xStart.append(pos[0]['x'])
        xEnd.append(pos[0]['x'])
        yStart.append(pos[0]['y'])
        yEnd.append(pos[0]['y'])

In [61]:

def Locations(event_df):

    xStart,xEnd,yStart,yEnd = [], [], [], []
    for pos in Events_England["positions"]:
        if len(pos) == 2:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[1]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[1]['y'])
        else:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[0]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[0]['y'])


    event_df['xStart'], event_df['xEnd'], event_df['yStart'], event_df['yEnd'] = xStart, xEnd, yStart, yEnd
    return event_df

In [62]:
Events_England = Locations(Events_England)

In [63]:
Events_England.drop(['eventId', 'positions', 'id', "event_tags", "tags", "eventId", "eventName"], inplace = True, axis = 1)

In [64]:
Events_England

,subEventName,playerId,matchId,teamId,matchPeriod,eventSec,subEventId,Result,subEventDescription,xStart,xEnd,yStart,yEnd
0,Simple pass,25413,2499719,1609,1H,2.758649,85,Success,generic play,49,31,49,78
1,High pass,370224,2499719,1609,1H,4.946850,83,Success,generic play,31,51,78,75
2,Head pass,3319,2499719,1609,1H,6.542188,82,Success,generic play,51,35,75,71
3,Head pass,120339,2499719,1609,1H,8.143395,82,Success,generic play,35,41,71,95
4,Simple pass,167145,2499719,1609,1H,10.302366,85,Success,generic play,41,72,95,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
643143,Cross,14703,2500098,1633,2H,2789.232666,80,Failure,right_foot blocked,99,0,72,0
643146,Corner,70965,2500098,1633,2H,2829.821084,30,Success,key_pass high,100,88,100,47
643147,Air duel,7919,2500098,1623,2H,2831.211419,10,Failure,lost,12,14,53,50
643148,Air duel,8005,2500098,1633,2H,2832.434399,10,Success,won,88,86,47,50


In [65]:
Matches.rename(columns={'wyId':'matchId'}, inplace=True)

In [288]:
test = pd.DataFrame(Events_England[Events_England["subEventName"]=="Corner"][['matchId', "teamId", "subEventName"]].groupby(['matchId', 'teamId', 'subEventName']).size())

In [292]:
Events_England[Events_England['matchId']==2500096]

,subEventName,playerId,matchId,teamId,matchPeriod,eventSec,subEventId,Result,subEventDescription,xStart,xEnd,yStart,yEnd
638400,Simple pass,25572,2500096,10531,1H,2.454344,85,Success,generic play,51,38,51,39
638401,Simple pass,8976,2500096,10531,1H,4.235506,85,Success,generic play,38,26,39,35
638402,Simple pass,246866,2500096,10531,1H,5.687350,85,Success,generic play,26,36,35,47
638403,Simple pass,8976,2500096,10531,1H,9.812816,85,Success,generic play,36,30,47,29
638404,Simple pass,246866,2500096,10531,1H,12.362791,85,Success,generic play,30,43,29,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
640177,Simple pass,77550,2500096,10531,2H,2693.805823,85,Success,generic play,61,55,30,35
640178,Simple pass,246866,2500096,10531,2H,2695.480439,85,Success,generic play,55,65,35,12
640179,Simple pass,8192,2500096,10531,2H,2699.216079,85,Success,generic play,65,72,12,14
640180,Ground defending duel,8731,2500096,1639,2H,2703.131853,12,Success,neutral,28,100,86,100


In [295]:
# Parsing out individual match statistics
test = pd.DataFrame(Events_England[Events_England["subEventName"]=="Corner"][['matchId', "teamId", "subEventName"]])
test = test.groupby(['matchId','teamId']).size().to_frame('Corners').reset_index()
g = test.groupby('matchId').agg(['unique']).reset_index()

In [296]:
g

,matchId,teamId,Corners
,,unique,unique
0,2499719,"[1609, 1631]","[9, 4]"
1,2499720,"[1625, 1651]","[10, 3]"
2,2499721,"[1610, 1646]","[8, 5]"
3,2499722,"[1628, 1673]","[12, 9]"
4,2499723,"[1623, 1639]","[6, 7]"
...,...,...,...
375,2500094,"[1610, 1613]","[2, 4]"
376,2500095,"[1619, 1625]","[1, 12]"
377,2500096,[10531],[6]


In [297]:
def missing_teams(x):
    if np.isnan(x['team2']):
        row = Matches[Matches['matchId'] == x["matchId"]]
        if int(row['homeTeamId']) == x['team1']:
            return int(row['awayTeamId'])
        else:
            return int(row['homeTeamId'])
    else:
        return x['team2']

In [305]:
f = pd.DataFrame(None, columns = ['matchId', 'teamId', "Corners"])

f['matchId'], f['teamId'], f['Corners'] = g.iloc[:, 0], g.iloc[:,1], g.iloc[:,2]


h = pd.concat([f, pd.DataFrame(f['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
h["team2"] = h.apply(lambda x : missing_teams(x), axis=1)

h = pd.concat([h, pd.DataFrame(f['Corners'].to_list(), columns=['cornersTeam1','cornersTeam2'])], axis=1, sort = False)
h.drop(['teamId', "Corners"], inplace = True, axis =1 )
h['cornersTeam2'] = h['cornersTeam2'].fillna(0)



,matchId,team1,team2,cornersTeam1,cornersTeam2
0,2499719,1609,1631.0,9,4.0
1,2499720,1625,1651.0,10,3.0
2,2499721,1610,1646.0,8,5.0
3,2499722,1628,1673.0,12,9.0
4,2499723,1623,1639.0,6,7.0
...,...,...,...,...,...
375,2500094,1610,1613.0,2,4.0
376,2500095,1619,1625.0,1,12.0
377,2500096,10531,1639.0,6,0.0
378,2500097,1624,1631.0,4,0.0


In [192]:
g = test.groupby('matchId')['teamId','Corners'].apply(lambda x: list(np.unique(x)))


<ipython-input-192-b35df3a9071e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  g = test.groupby('matchId')['teamId','Corners'].apply(lambda x: list(np.unique(x)))


In [196]:
pd.DataFrame(g).reset_index()

,matchId,0
0,2499719,"[4, 9, 1609, 1631]"
1,2499720,"[3, 10, 1625, 1651]"
2,2499721,"[5, 8, 1610, 1646]"
3,2499722,"[9, 12, 1628, 1673]"
4,2499723,"[6, 7, 1623, 1639]"
...,...,...
375,2500094,"[2, 4, 1610, 1613]"
376,2500095,"[1, 12, 1619, 1625]"
377,2500096,"[6, 10531]"
378,2500097,"[4, 1624, 1631]"


In [71]:
Matches

,winner,venue,matchId,date,competitionId,Country,homeScore,awayScore,homeScoreHT,awayScoreHT,...,awayBenchPlayer4,awayBenchPlayer5,awayBenchPlayer6,awayBenchPlayer7,homeManager,homeTeamId,awayManager,awayTeamId,Result,time
0,1659,Turf Moor,2500089,2018-05-13,364,England,1,2,1,0,...,515791,7989.0,8296.0,11669.0,8880,1646,8934,1659,-1,14:00:00
1,1628,Selhurst Park,2500090,2018-05-13,364,England,2,0,0,0,...,8096,7907.0,261.0,8686.0,8357,1628,0,1627,1,14:00:00
2,1609,The John Smith's Stadium,2500091,2018-05-13,364,England,0,1,0,1,...,171166,7945.0,3560.0,279717.0,18572,1673,7845,1609,-1,14:00:00
3,1612,Anfield,2500092,2018-05-13,364,England,4,0,2,0,...,4255,15526.0,8416.0,13451.0,14791,1612,8093,1651,1,14:00:00
4,1611,Old Trafford,2500093,2018-05-13,364,England,1,0,1,0,...,234408,8889.0,7888.0,8903.0,3295,1611,93112,1644,1,14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,9598,Parc des Princes,1694408,2016-06-12,102,European Champs,0,1,0,1,...,69417,284470.0,69404.0,284315.0,32637,4687,69952,9598,-1,13:00:00
1937,0,Stade V\u00e9lodrome,1694397,2016-06-11,102,European Champs,1,1,0,0,...,101663,102083.0,101699.0,101704.0,8357,2413,101575,14358,0,19:00:00
1938,10682,Stade Matmut-Atlantique,1694396,2016-06-11,102,European Champs,2,1,1,0,...,101555,138408.0,32803.0,101813.0,136439,10682,268998,14496,1,16:00:00
1939,6697,Stade Bollaert-Delelis,1694391,2016-06-11,102,European Champs,0,1,0,1,...,360992,49919.0,134911.0,236528.0,135480,8731,210701,6697,-1,13:00:00


In [69]:
cols = {'matchId': [None, None], 'Corners1': [None, None]}
match_stats = pd.DataFrame(data=cols)

In [70]:
match_stats['matchId']

,matchId,Corners1
0,None,None
1,None,None


In [67]:
### Build function for adding aggregations to matches

def aggreg_match_stats(event_df, Matches_df):
    





SyntaxError: unexpected EOF while parsing (<ipython-input-67-04dafd13537d>, line 7)

### Attempt to design a "danger index" for each action and then aggregate for players and teams
##### Should also contemplate design for chemistry index, and Goalproof index

In [ ]:
# (-15)**2*-0.05

In [ ]:
# (trial['xEnd']- trial['xStart'])/100

In [ ]:
# trial["Danger_index"] = (trial['yEnd']**2) * (trial['xEnd']- trial['xStart'])/100

In [ ]:
# trial[0:3]

In [ ]:
trial[(trial['subEventName'] == "Shot") & (trial['Result']=="Failure")]

In [ ]:
trial[0:2]['EndLoc'][0]['y'] == 100

In [ ]:
Goal = []
for index, row in trial.iterrows():
    if 'goal' in row['subEventDescription'] and row['EndLoc']['x'] == 100:
        Goal.append(1)
    else:
        Goal.append(0)
trial["ScoresGoal"] = Goal

In [ ]:
trial

In [ ]:
# EDIT MATCH TIME COLUMN

In [ ]:
try:
    os.chdir('/Users/jordanmakins/Desktop/Data/events')
except:
    pass
Events_England = pd.read_json('events_England.json')

In [ ]:
trial2 = Events_England[0:4000]

In [ ]:
# Solution to convert second half time into 
secs_to_add = list(trial2[trial2['matchPeriod']=="1H"].groupby('matchId').tail(1)['eventSec'])
match_ids = list(trial2["matchId"].unique())

for idx in range(0, len(match_ids)):
    trial2['eventSec'] = list(np.where(
       (trial2['matchId'] == match_ids[idx]) & (trial2['matchPeriod'] == "2H") , trial2['eventSec'] + secs_to_add[idx], trial2['eventSec']
       ))
    

In [ ]:
# Add a column for euc

### First Attempt with Hugo Mathien

In [ ]:
# cursor = sql_conn.cursor()
# table_names = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

In [ ]:
# sql_conn2 = sqlite3.connect('hugomathienData.sqlite') # soccer database containing over 100,000 match results which is updated monthly
# cursor2 = sql_conn2.cursor()
# table_names = cursor2.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor2.fetchall())

In [ ]:
# Country = pd.read_sql_query("SELECT * FROM Country", sql_conn2)
# League = pd.read_sql_query("SELECT * FROM League", sql_conn2)
# Match_hugo = pd.read_sql_query("SELECT * FROM Match", sql_conn2)
# Player = pd.read_sql_query("SELECT * FROM Player", sql_conn2)
# Player_Attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", sql_conn2)
# Team = pd.read_sql_query("SELECT * FROM Team", sql_conn2)
# Team_Attributes = pd.read_sql_query("SELECT * FROM Team_Attributes", sql_conn2)

### Data Processing

In [ ]:
#Match_TeamStats = pd.read_sql_query("SELECT * FROM football_data", sql_conn)

In [ ]:
# # parsing betting odds variables from analysis
# drop_columns = ["B365H","B365D","B365A","BSH","BSD","BSA","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA",
# "PSH", "PH","PSD", "PD","PSA", "PA","SOH","SOD","SOA","SBH","SBD","SBA","SJH","SJD","SJA","SYH","SYD","SYA","VCH",
# "VCD","VCA","WHH","WHD","WHA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","MaxH","MaxD","MaxA",
# "AvgH","AvgD","AvgA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","GB>2.5","GB<2.5","B365>2.5","B365<2.5",
# "P>2.5","P<2.5","Max>2.5","Max<2.5","Avg>2.5","Avg<2.5","BbAH","BbAHh","AHh","BbMxAHH","BbAvAHH","BbMxAHA",
# "BbAvAHA","GBAHH","GBAHA","GBAH","LBAHH","LBAHA","LBAH","B365AHH","B365AHA","B365AH","PAHH", "AvgC<2.5",
# "PAHA","MaxAHH","MaxAHA","AvgAHH","AvgAHA","WHCH","WHCD","WHCA", "VCCH", "VCCA", "PSCH", "PSCD", "PCAHH", "PCAHA", "PC>2.5", "PC<2.5", "MaxCH", "MaxCAHH", "MaxCD",
# "MaxCAHA", "MaxCA", "MaxC>2.5", "MaxC<2.5", "IWCH", "IWCD", "IWCA", "AvgC>2.5","AvgCA","AvgCAHA","AvgCAHH",
# "AvgCD","AvgCH","B365C<2.5","B365C>2.5","B365CA","B365CAHA","B365CAHH","B365CD","B365CH","BWCA", "BWCD", "BWCH", "VCCD", "PSCA"] 

In [ ]:
# Match_TeamStats.drop(drop_columns, inplace=True, axis =1) # removing betting odds from dataframe

In [ ]:
def df_analysis(df, df_name):
    header = "Dimension of " + str(df_name) + " Dataframe is: {}"
    print(header.format(df.shape))
    print(100*"*")
    print(df.info())
    print(100*"*")
    print(df.select_dtypes(exclude=['float64','int64']).describe())
    print(100*"*")
    print(df.describe())
    print(100*"*")
    print("Number of Null Values")
    print(df.isnull().sum(axis=0))

In [ ]:
# df_analysis(Match_TeamStats, "Match_TeamStats")

In [ ]:
# match[match["HomeTeam"].str.contains("Man United")]

### Outlier Detection and Understanding